In [2]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import pickle
import os
from time import time
import pandas as pd
import numpy as np
from mylib.tokenizer import tokenizer, tokenizer_porter
import nltk

github_path = 'C:/Users/taehee/Documents/GitHub/tripReviewAnalysisSystem/'
#engine='python' 을 넣어줘야 판다스가 한국어로 된 폴더명을 인식한다.

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


In [3]:
def model_tfidf(df, train_size, c_value):
    '''
    입력 : text와 sentiment가 포함된 리뷰들의 데이터 프레임
    출력 : 훈련이 끝난 분류기와 정확도를 리턴.
    '''

    X = df.loc[:, 'text'].values
    y = df.loc[:, 'sentiment'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                        test_size=1 - train_size,
                                                        random_state=14,
                                                        stratify=y)

    tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer)
    Ir_tfidf = Pipeline([('vect', tfidf), ('clf', LogisticRegression(C=c_value, penalty='l2', random_state=0, solver='liblinear'))])

    Ir_tfidf.fit(X_train, y_train)
    y_pred = Ir_tfidf.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    print('정확도: %.3f' % test_accuracy)

    return Ir_tfidf, test_accuracy



In [4]:
def all_reviews(github_path, file_list, size):
    result = pd.read_csv(github_path + '크롤러-전처리/원시자료/' + file_list[0], encoding='utf-8', engine='python', index_col=0)
    stime = time()
    print('데이터 로딩 start')

    for file_path in file_list[:size]:
        new_data = pd.read_csv(github_path + '크롤러-전처리/원시자료/' + file_path, encoding='utf-8', engine='python',
                               index_col=0)
        # 새 데이터를 리스트로 합쳐서 concat시키면 행이 계속 추가 된다.
        # ignore_index로 기존 인덱스를  무시하고 합친 결과를 다시 인덱스해준다. 이거 안 하면 index must be monotonic increasing or decreasing 에러가 뜰 것이다.
        result = pd.concat([result, new_data], ignore_index=True, sort=False)

    print('데이터 로딩 종료: 소요시간 [%d]초' % (time() - stime))

    result['sentiment'] = np.where(result.star_point > 3, '1', '0')

    return result

In [5]:
# 리뷰 리스트 불러오기기
file_list = os.listdir(github_path + '크롤러-전처리/원시자료/')

print("file_list: {}".format(file_list[:10]))

result = pd.read_csv(github_path + '크롤러-전처리/원시자료/' + file_list[0], encoding='utf-8', engine='python')
result.head()

file_list: ['63 City.txt', 'Alive Museum Insadong.txt', 'Amore Pacific Museum of Art.txt', 'An Jung-geun Memorial Hall.txt', 'Ansan Jarak-gil.txt', 'apm Place.txt', 'Arario Museum in Space.txt', "Baek In-je's House.txt", 'Banpodaegyo Bridge.txt', 'Bongeunsa Temple.txt']


,Unnamed: 0,star_point,title,text,Date of experience
0,0,3.0,nothing special,just a tall building in a residential area. no...,February 2020
1,1,4.0,City Views,"Had lunch on 59th floor, outstanding views of ...",October 2019
2,2,3.0,Great Views,Stopped by here as it was along my travel plan...,April 2019
3,3,3.0,Just a tall building,I don't know what I was expecting but this was...,August 2018
4,4,3.0,63 City,NaN,July 2019


In [13]:
data_all = all_reviews(github_path, file_list, len(file_list))

데이터 로딩 start
데이터 로딩 종료: 소요시간 [1]초


In [14]:
data_all.isnull().sum()

star_point               0
title                    3
text                  3657
Date of experience       0
sentiment                0
dtype: int64

In [ ]:
model = model_tfidf(data_all, train_size=0.7, c_value=1.0)